<a href="https://colab.research.google.com/github/Edfred1/Contextual-Music-Crafter/blob/main/CMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation and Setup

This section covers cloning the repository and installing the necessary dependencies.

In [ ]:
# Clone the Git repository
!git clone https://github.com/Edfred1/Contextual-Music-Crafter.git
%cd Contextual-Music-Crafter

# Install dependencies
!pip install -r requirements.txt

### Configuration

Here, we configure the `config.yaml` file. This file contains settings for the AI API, musical parameters, and instrument definitions. You'll need to replace `"YOUR_GOOGLE_AI_API_KEY"` with your actual Google AI API key.

In [ ]:
from ruamel.yaml import YAML
from pathlib import Path

# Path to config.yaml
config_path = Path('config.yaml')

# Load existing config (preserve comments); create a minimal starter if missing
yaml = YAML(typ='rt')
yaml.preserve_quotes = True
yaml.indent(mapping=2, sequence=4, offset=2)

doc = None
if config_path.exists():
    try:
        with config_path.open('r', encoding='utf-8') as f:
            doc = yaml.load(f)
    except Exception:
        doc = None

if doc is None:
    doc = yaml.load("""
api_key:
  - "YOUR_GOOGLE_AI_API_KEY_1"
model_name: "gemini-2.5-pro"
temperature: 0
lyrics_temperature: 0.0
enable_hotkeys: 1
stage2_invalid_retries: 5
pass_raw_prompt_to_stages: 0
inspiration: "A new track"
genre: "House"
bpm: 125
key_scale: "C minor"
automation_settings:
  use_pitch_bend: 0
  use_sustain_pedal: 0
  use_cc_automation: 0
  allowed_cc_numbers: [1, 10, 11, 74]
max_output_tokens: 65536
context_window_size: -1
use_call_and_response: 0
number_of_iterations: 1
time_signature:
  beats_per_bar: 4
  beat_value: 4
instruments:
  - name: "Drums"
    program_num: 10
    role: "drums"
  - name: "Bass"
    program_num: 39
    role: "bass"
""")

# Helper function to convert to 0/1
def _to01(x):
    try:
        return 1 if int(x) == 1 else 0
    except Exception:
        return 0

# Helper function to convert to int
def _toint(x, default):
    try:
        return int(x)
    except Exception:
        return default

# Helper function to convert to float
def _tofloat(x, default):
    try:
        return float(x)
    except Exception:
        return default

print("=== API Configuration ===")
# API keys
api_keys_input = input(f"Enter Google AI API key(s), comma-separated (Enter to keep current): ").strip()
if api_keys_input:
    keys = [k.strip() for k in api_keys_input.split(',') if k.strip()]
    doc['api_key'] = keys

# Model name
model = input(f"Model name [{doc.get('model_name','gemini-2.5-pro')}]: ").strip()
if model:
    doc['model_name'] = model

# Temperature
temp = input(f"Temperature [{doc.get('temperature',0)}]: ").strip()
if temp:
    doc['temperature'] = _tofloat(temp, doc.get('temperature',0))

# Lyrics temperature
lyrics_temp = input(f"Lyrics temperature [{doc.get('lyrics_temperature',0.0)}]: ").strip()
if lyrics_temp:
    doc['lyrics_temperature'] = _tofloat(lyrics_temp, doc.get('lyrics_temperature',0.0))

# Enable hotkeys
hotkeys = input(f"Enable hotkeys (1=on, 0=off) [{doc.get('enable_hotkeys',1)}]: ").strip()
if hotkeys:
    doc['enable_hotkeys'] = _to01(hotkeys)

# Stage 2 invalid retries
retries = input(f"Stage-2 invalid retries [{doc.get('stage2_invalid_retries',5)}]: ").strip()
if retries:
    doc['stage2_invalid_retries'] = _toint(retries, doc.get('stage2_invalid_retries',5))

# Pass raw prompt to stages
raw_prompt = input(f"Pass raw prompt to stages (1=yes, 0=no) [{doc.get('pass_raw_prompt_to_stages',0)}]: ").strip()
if raw_prompt:
    doc['pass_raw_prompt_to_stages'] = _to01(raw_prompt)

print("\n=== Musical Parameters ===")
# Inspiration
insp = input(f"Inspiration [{doc.get('inspiration','A new track')}]: ").strip()
if insp:
    doc['inspiration'] = insp

# Genre
genre = input(f"Genre [{doc.get('genre','House')}]: ").strip()
if genre:
    doc['genre'] = genre

# BPM
bpm = input(f"BPM [{doc.get('bpm',125)}]: ").strip()
if bpm:
    doc['bpm'] = _toint(bpm, doc.get('bpm',125))

# Key and scale
key_scale = input(f"Key and scale [{doc.get('key_scale','C minor')}]: ").strip()
if key_scale:
    doc['key_scale'] = key_scale

print("\n=== Automation Settings ===")
auto = doc.get('automation_settings') or {}
if input("Edit automation settings? (y/N): ").strip().lower() == 'y':
    auto['use_pitch_bend'] = _to01(input(f"use_pitch_bend (1=on, 0=off) [{auto.get('use_pitch_bend',0)}]: ") or auto.get('use_pitch_bend',0))
    auto['use_sustain_pedal'] = _to01(input(f"use_sustain_pedal (1=on, 0=off) [{auto.get('use_sustain_pedal',0)}]: ") or auto.get('use_sustain_pedal',0))
    auto['use_cc_automation'] = _to01(input(f"use_cc_automation (1=on, 0=off) [{auto.get('use_cc_automation',0)}]: ") or auto.get('use_cc_automation',0))
    if auto.get('use_cc_automation',0) == 1:
        cc_str = input(f"allowed_cc_numbers (comma-separated) [{auto.get('allowed_cc_numbers',[1,10,11,74])}]: ").strip()
        if cc_str:
            try:
                auto['allowed_cc_numbers'] = [int(x.strip()) for x in cc_str.split(',') if x.strip()]
            except Exception:
                pass
doc['automation_settings'] = auto

print("\n=== Generation & Performance Settings ===")
# Max output tokens
max_tokens = input(f"Max output tokens [{doc.get('max_output_tokens',65536)}]: ").strip()
if max_tokens:
    doc['max_output_tokens'] = _toint(max_tokens, doc.get('max_output_tokens',65536))

# Context window size
ctx_size = input(f"Context window size (-1=dynamic, 0=none, >0=fixed) [{doc.get('context_window_size',-1)}]: ").strip()
if ctx_size:
    doc['context_window_size'] = _toint(ctx_size, doc.get('context_window_size',-1))

# Use call and response
call_resp = input(f"Use call and response (1=yes, 0=no) [{doc.get('use_call_and_response',0)}]: ").strip()
if call_resp:
    doc['use_call_and_response'] = _to01(call_resp)

# Number of iterations
iterations = input(f"Number of iterations [{doc.get('number_of_iterations',1)}]: ").strip()
if iterations:
    doc['number_of_iterations'] = _toint(iterations, doc.get('number_of_iterations',1))

# Time signature
print("\n=== Time Signature ===")
ts = doc.get('time_signature') or {}
beats_per_bar = input(f"Beats per bar [{ts.get('beats_per_bar',4)}]: ").strip()
if beats_per_bar:
    ts['beats_per_bar'] = _toint(beats_per_bar, ts.get('beats_per_bar',4))
beat_value = input(f"Beat value [{ts.get('beat_value',4)}]: ").strip()
if beat_value:
    ts['beat_value'] = _toint(beat_value, ts.get('beat_value',4))
doc['time_signature'] = ts

# Instruments
print("\n=== Instruments ===")
if input("Edit instruments? (y/N): ").strip().lower() == 'y':
    instruments = doc.get('instruments', [])
    print(f"Current instruments: {len(instruments)}")
    for i, inst in enumerate(instruments):
        print(f"  {i+1}. {inst.get('name','Unknown')} - Program {inst.get('program_num',0)} - Role: {inst.get('role','unknown')}")
    
    action = input("Add new instrument? (y/N): ").strip().lower()
    if action == 'y':
        name = input("Instrument name: ").strip()
        if name:
            program = input("Program number (1-128): ").strip()
            role = input("Role (drums, bass, pads, lead, melody, etc.): ").strip()
            if program and role:
                try:
                    new_inst = {
                        'name': name,
                        'program_num': _toint(program, 1),
                        'role': role
                    }
                    instruments.append(new_inst)
                    doc['instruments'] = instruments
                    print(f"Added instrument: {name}")
                except Exception as e:
                    print(f"Error adding instrument: {e}")
    
    # Option to remove instruments
    if instruments:
        remove = input("Remove instrument? (y/N): ").strip().lower()
        if remove == 'y':
            try:
                idx = int(input(f"Enter instrument number to remove (1-{len(instruments)}): ").strip()) - 1
                if 0 <= idx < len(instruments):
                    removed = instruments.pop(idx)
                    doc['instruments'] = instruments
                    print(f"Removed instrument: {removed.get('name','Unknown')}")
            except Exception:
                print("Invalid input.")

# Save back preserving comments
with config_path.open('w', encoding='utf-8') as f:
    yaml.dump(doc, f)

print("\nconfig.yaml updated (comments preserved).")

### Running a Python Program from the Repository

This cell lists the Python programs found in the cloned repository and prompts you to select one to execute. The selected program will then be run based on the configuration.

In [ ]:
import os

# Get a list of all files and directories in the cloned repository
files_and_directories = os.listdir('.')

# Filter for Python files
python_files = [f for f in files_and_directories if f.endswith('.py')]

if not python_files:
    print("No Python files found in the repository.")
else:
    print("Found Python files:")
    for i, filename in enumerate(python_files):
        print(f"{i+1}. {filename}")

    # Prompt the user to select a file
    while True:
        try:
            choice = int(input(f"Enter the number of the file to execute (1-{len(python_files)}): "))
            if 1 <= choice <= len(python_files):
                selected_file = python_files[choice - 1]
                print(f"Executing {selected_file}...")
                # Execute the selected Python file
                !python {selected_file}
                break
            else:
                print("Invalid input. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")